In [2]:
import pyaudio
import wave
import os
import threading

def list_input_devices():
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    num_devices = info.get('deviceCount')
    devices = []

    for i in range(num_devices):
        device_info = p.get_device_info_by_host_api_device_index(0, i)
        devices.append(device_info['name'])

    p.terminate()
    return devices

def select_input_device(devices):
    print("Available input devices:")
    for i, device in enumerate(devices):
        print(f"{i+1}. {device}")
    
    selection = int(input("Select an input device (enter the number): ")) - 1
    if selection < 0 or selection >= len(devices):
        print("Invalid selection.")
        return None
    return selection

def record_audio(file_name, input_device_index, sample_rate=44100, channels=1):
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    input_device_index=input_device_index,
                    frames_per_buffer=1024)

    print("Recording... Press Enter to stop.")

    frames = []
    stop_event = threading.Event()  # Event to signal the recording to stop

    def record_thread():
        nonlocal frames
        while not stop_event.is_set():
            data = stream.read(1024)
            frames.append(data)

    record_thread = threading.Thread(target=record_thread)
    record_thread.start()

    input("Press Enter to stop recording...")
    stop_event.set()  # Set the event to stop recording

    record_thread.join()

    print("Recording finished.")

    stream.stop_stream()
    stream.close()

    p.terminate()

    if not file_name.lower().endswith('.wav'):
        file_name += '.wav'

    wf = wave.open(file_name, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

if __name__ == "__main__":
    input_devices = list_input_devices()

    if not input_devices:
        print("No input devices found.")
    else:
        print("Select an input device:")
        selected_device_index = select_input_device(input_devices)
        if selected_device_index is not None:
            output_folder = "record"  # Define the output folder
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)  # Create the folder if it doesn't exist

            output_file = input("Enter the output file name (e.g., recorded_audio): ")

            output_path = os.path.join(output_folder, output_file)  # New line: Construct the output path
            record_audio(output_path, selected_device_index)


Select an input device:
Available input devices:
1. LG ULTRAWIDE
2. 갤럭시 S2 마이크
3. USB PnP Sound Device
4. 외장 마이크
5. 외장 헤드폰
6. MacBook Air 마이크
7. MacBook Air 스피커


ValueError: invalid literal for int() with base 10: ''